In [105]:
import pandas as pd
import os
import FinanceDataReader as fdr
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from pmdarima.arima import auto_arima

In [106]:
path = './'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list = stock_list.sort_values("종목코드")

In [107]:
data = pd.DataFrame()
volume = pd.DataFrame()
high = pd.DataFrame()
low = pd.DataFrame()
data2 = pd.DataFrame()
volume2 = pd.DataFrame()
high2 = pd.DataFrame()
low2 = pd.DataFrame()

start_date = '20160104'
end_date = '20200214'
start_date2 = '20200608'
end_date2 = '20211126'

for stock_code in stock_list["종목코드"]:
    stock = fdr.DataReader(stock_code, start = start_date, end = end_date)
    data = pd.concat([data,stock["Close"]], axis=1)
    volume = pd.concat([volume, stock["Volume"]], axis=1)
    high = pd.concat([high,stock["High"]], axis=1)
    low = pd.concat([low,stock["Low"]], axis=1)

for stock_code in stock_list["종목코드"]:
    stock = fdr.DataReader(stock_code, start = start_date2, end = end_date2)
    data2 = pd.concat([data2,stock["Close"]], axis=1)
    volume2 = pd.concat([volume2, stock["Volume"]], axis=1)
    high2 = pd.concat([high2,stock["High"]], axis=1)
    low2 = pd.concat([low2,stock["Low"]], axis=1)

data = pd.concat([data,data2], axis=0)
volume = pd.concat([volume, volume2], axis=0)
high = pd.concat([high,high2], axis=0)
low = pd.concat([low,low2], axis=0)

data.columns = stock_list["종목코드"]
volume.columns = stock_list["종목코드"]
high.columns = stock_list["종목코드"]
low.columns = stock_list["종목코드"]

In [108]:
from pmdarima.arima import auto_arima
data_diff = data.iloc[1:, :] - data.iloc[:-1, :].values

def dr_outlier(df): 
    quartile_1 = df.quantile(0.25) 
    quartile_3 = df.quantile(0.75) 
    IQR = quartile_3 - quartile_1 
    condition = (df < (quartile_1 - 1.5 * IQR)) | (df > (quartile_3 + 1.5 * IQR)) 
    search_df = df[condition] 
    return df.drop(search_df.index, axis=0)

from pmdarima.arima import ndiffs
from sklearn.linear_model import LinearRegression

def arima_pred_model(train):
    if train.isnull().sum() > 0:
        train = train.dropna(axis=0)
    kpss_diffs = ndiffs(train, alpha=0.1, test='kpss', max_d=6)
    adf_diffs = ndiffs(train, alpha=0.1, test='adf', max_d=6)
    n_diffs = max(adf_diffs, kpss_diffs)
    m = auto_arima(train, d=n_diffs, error_action='ignore', start_p=1, start_q=1, max_p=3, max_q=3)
    pred = m.predict(n_periods=5)
    return pred

result = []
for index in tqdm(range(370)):
    train = data_diff.iloc[:, index]
    train = dr_outlier(train)
    pred = arima_pred_model(train)
    result.append(pred)

sub = pd.read_csv("sample_submission.csv")
result = pd.DataFrame(result).T
for i in range(1,371):
    for j in range(5):
        result_sum = result.iloc[:j+1, i-1].sum()
        sub.iloc[j + 5, i] = np.pad(result_sum + data.iloc[-1, i - 1], (5, 0), 'constant', constant_values=0)
sub["031390"] = 0
sub.to_csv("arima_final2.csv", index=0)

In [ ]:
'''2658.6630261375826 2623.8937971427777 2712.1626005471353 2702.7738540248515
'''
